In [1]:
import numpy
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
import matplotlib as plt
plt.rcParams["figure.figsize"] = (10, 5)
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
# warnings.filterwarnings('ignore')
# import pycountry
sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua
import numba
import copy
import glob
from numba import njit
import pandas
import glob
import pickle
import multiprocessing
from functools import partial

/tmp/ipykernel_3450914/564853260.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## 1 station 

In [2]:
a = eua.vm_request_wrapper({'variable': 'temperature', 'statid': ['11035']}, vm_url='http://localhost:8004',overwrite=True)
a.to_dataframe()[10000:10010]

,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
10000,48.32,16.120001,0,85000.0,20000000846,WIEN/HOHE WARTE (11035-1),290.750000,1950-08-25 21:00:00,nnnnnnnnnnn
10001,48.32,16.120001,0,90000.0,20000000846,WIEN/HOHE WARTE (11035-1),292.549988,1950-08-25 21:00:00,nnnnnnnnnnn
10002,48.32,16.120001,0,95000.0,20000000846,WIEN/HOHE WARTE (11035-1),293.549988,1950-08-25 21:00:00,nnnnnnnnnnn
10003,48.32,16.120001,0,99800.0,20000000846,WIEN/HOHE WARTE (11035-1),292.549988,1950-08-25 21:00:00,nnnnnnnnnnn
10004,48.32,16.120001,0,12500.0,20000000847,WIEN/HOHE WARTE (11035-1),220.149994,1950-08-26 03:00:00,nnnnnnnnnnn
10005,48.32,16.120001,0,15000.0,20000000847,WIEN/HOHE WARTE (11035-1),222.649994,1950-08-26 03:00:00,nnnnnnnnnnn
10006,48.32,16.120001,0,17500.0,20000000847,WIEN/HOHE WARTE (11035-1),219.750000,1950-08-26 03:00:00,nnnnnnnnnnn
10007,48.32,16.120001,0,20000.0,20000000847,WIEN/HOHE WARTE (11035-1),220.949997,1950-08-26 03:00:00,nnnnnnnnnnn
10008,48.32,16.120001,0,25000.0,20000000847,WIEN/HOHE WARTE (11035-1),229.649994,1950-08-26 03:00:00,nnnnnnnnnnn
10009,48.32,16.120001,0,30000.0,20000000847,WIEN/HOHE WARTE (11035-1),238.649994,1950-08-26 03:00:00,nnnnnnnnnnn


In [3]:
a = eua.vm_request_wrapper({'variable': 'temperature', 'statid': ['11035'], 'fast_csv':'True'}, vm_url='http://localhost:8004',overwrite=True)
a[10000:10010]

,# latitude,longitude,date_time,z_coordinate,observation_value
10000,48.32,16.120001,1598302800,85000.0,290.750000
10001,48.32,16.120001,1598302800,90000.0,292.549988
10002,48.32,16.120001,1598302800,95000.0,293.549988
10003,48.32,16.120001,1598302800,99800.0,292.549988
10004,48.32,16.120001,1598324400,12500.0,220.149994
10005,48.32,16.120001,1598324400,15000.0,222.649994
10006,48.32,16.120001,1598324400,17500.0,219.750000
10007,48.32,16.120001,1598324400,20000.0,220.949997
10008,48.32,16.120001,1598324400,25000.0,229.649994
10009,48.32,16.120001,1598324400,30000.0,238.649994


### csv

In [4]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'statid': ['11035'], 'fast_csv':'True', 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

17.286755323410034


#### zipping: 0.4437854290008545 sec


### nc

In [5]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'statid': ['11035'], 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

7.595234632492065


#### zipping: 0.3818824291229248 sec

## 1 month all stations

### csv

In [6]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'date':['20100101','20100201'], 'fast_csv':'True', 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

2.9086570739746094


#### zipping: 0.22829532623291016 sec

### nc

In [7]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'date':['20100101','20100201'], 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

4.315251350402832


#### zipping: 0.23750019073486328 sec

## 300 hPa all stations

### csv

In [10]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'pressure_level':[30000], 'fast_csv':'True', 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

114.5682692527771


#### zipping: 3.1846954822540283 sec

### nc

In [11]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'pressure_level':[30000], 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

111.29002141952515


#### zipping: 3.222466230392456 sec